# PREAMBLE
<script
  src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML"
  type="text/javascript">
</script>

In [ ]:

import numpy as np
import pandas as pd
from validphys.api import API

%matplotlib inline

# Definition of the input

In [ ]:
fitname = "241107-02-rs-nnlo_mhou_qed-alphas-tcm-rep0"

# COMPUTATION OF $\alpha_s$

In [ ]:
fit = API.fit(fit=fitname)

prior_pdf = fit.as_input()["theorycovmatconfig"]["pdf"]

pps = fit.as_input()["theorycovmatconfig"]["point_prescriptions"]
alphas_pp_id, alphas_pp = [[j,i] for j,i in enumerate(pps) if "alphas" in i][0]

common_dict = dict(
    dataset_inputs={"from_": "fit"},
    fit=fit.name,
    fits=[fit.name],
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
)

theoryids_dict = ({
        "point_prescription": alphas_pp,
        "theoryid": {"from_": "theory"},
        "theory": {"from_": "fit"},
        "theorycovmatconfig": {"from_": "fit"},
    } | common_dict)

theoryids = API.theoryids(**theoryids_dict)
theory_plus = theoryids[1].id
theory_mid = theoryids[0].id
theory_min = theoryids[2].id

# Inputs for central theory (used to construct the alphas covmat)
inps_central = dict(theoryid=theory_mid, pdf=prior_pdf, **common_dict)

# Inputs for plus theory (used to construct the alphas covmat)
inps_plus = dict(theoryid=theory_plus, pdf=prior_pdf, **common_dict)

# Inputs for minus theory prediction (used to construct the alphas covmat)
inps_minus = dict(theoryid=theory_min, pdf=prior_pdf, **common_dict)

# inputs for the computation of the prediction of the fit with cov=C+S, where S is computed using the
# inps_central, inps_plus, and inps_minus dictionaries
inps_central_fit = dict(theoryid=theory_mid, pdf={"from_": "fit"}, **common_dict)

In [ ]:
prior_theorypreds_central = API.group_result_central_table_no_table(**inps_central)["theory_central"]
# prior_theorypreds_central = API.group_result_table_no_table(**inps_central).iloc[:, 2:].mean(axis=1)

In [ ]:
prior_theorypreds_plus = API.group_result_central_table_no_table(**inps_plus)["theory_central"]
# prior_theorypreds_plus = API.group_result_table_no_table(**inps_plus).iloc[:, 2:].mean(axis=1)

In [ ]:
prior_theorypreds_minus = API.group_result_central_table_no_table(**inps_minus)["theory_central"]
# prior_theorypreds_minus = API.group_result_table_no_table(**inps_minus).iloc[:, 2:].mean(axis=1)

In [ ]:
# Get the values of alphas...
alphas_plus = API.theory_info_table(theory_db_id=theory_plus).loc["alphas"].iloc[0]
alphas_central = API.theory_info_table(theory_db_id=theory_mid).loc["alphas"].iloc[0]
alphas_min = API.theory_info_table(theory_db_id=theory_min).loc["alphas"].iloc[0]

# ... and make sure the alphas shift in both directions is symmetric
delta_alphas_plus = alphas_plus - alphas_central
delta_alphas_min = alphas_central - alphas_min
if abs(delta_alphas_min - delta_alphas_plus) > 1e-6:
    raise ValueError("alphas shifts in both directions is not symmetric")
else:
    alphas_step_size = delta_alphas_min

In [ ]:
covmat_scaling_factor = fit.as_input().get("theorycovmatconfig",{}).get("rescale_alphas_covmat",1.0)

beta_tilde = np.sqrt(covmat_scaling_factor) * (alphas_step_size / np.sqrt(2)) * np.array([1, -1])
S_tilde = beta_tilde @ beta_tilde

In [ ]:
delta_plus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_plus - prior_theorypreds_central
)
delta_minus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_minus - prior_theorypreds_central
)

beta = [delta_plus, delta_minus]
S_hat = beta_tilde @ beta

S = np.outer(delta_plus, delta_plus) + np.outer(delta_minus, delta_minus)
S = pd.DataFrame(S, index=delta_minus.index, columns=delta_minus.index)

In [ ]:
stored_alphas_covmat = pd.read_csv(
    fit.path / f"tables/datacuts_theory_theorycovmatconfig_point_prescriptions{alphas_pp_id}_theory_covmat_custom_per_prescription.csv",
    index_col=[0, 1, 2],
    header=[0, 1, 2],
    sep="\t|,",
    encoding="utf-8",
    engine="python",
).fillna(0)
storedcovmat_index = pd.MultiIndex.from_tuples(
    [(aa, bb, np.int64(cc)) for aa, bb, cc in stored_alphas_covmat.index],
    names=["group", "dataset", "id"],
)  # make sure theoryID is an integer, same as in S
stored_alphas_covmat = pd.DataFrame(
    stored_alphas_covmat.values, index=storedcovmat_index, columns=storedcovmat_index
)
stored_alphas_covmat = stored_alphas_covmat.reindex(S.index).T.reindex(S.index)

if not np.allclose(S, stored_alphas_covmat):
    print("Reconstructed theory covmat, S, is not the same as the stored covmat!")

In [ ]:
theorypreds_fit = API.group_result_table_no_table(**inps_central_fit).iloc[:, 2:]

In [ ]:
C = API.groups_covmat(
    use_t0=True,
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
    theoryid= {"from_": "theory"},
    theory={"from_": "fit"},
    **common_dict
)

In [ ]:
total_th_covmat = pd.read_csv(
    fit.path / f"tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv",
    index_col=[0, 1, 2],
    header=[0, 1, 2],
    sep="\t|,",
    encoding="utf-8",
    engine="python",
).fillna(0)
storedcovmat_index = pd.MultiIndex.from_tuples(
    [(aa, bb, np.int64(cc)) for aa, bb, cc in total_th_covmat.index],
    names=["group", "dataset", "id"],
)  # make sure theoryID is an integer, same as in S
total_th_covmat = pd.DataFrame(
    total_th_covmat.values, index=storedcovmat_index, columns=storedcovmat_index
)
total_th_covmat = total_th_covmat.reindex(S.index).T.reindex(S.index)

# we don't distinguish between exp uncs and theory uncs other than alphas, so add them here
C += total_th_covmat - stored_alphas_covmat

In [ ]:
# Different from the prediction of the mean PDF (i.e. replica0)
mean_prediction = theorypreds_fit.mean(axis=1)

X = np.zeros_like(C.values)
for i in range(theorypreds_fit.shape[1]):
    X += np.outer(
        (theorypreds_fit.iloc[:, i] - mean_prediction),
        (theorypreds_fit.iloc[:, i] - mean_prediction),
    )
X *= 1 / theorypreds_fit.shape[1]

In [ ]:
pseudodata = API.read_pdf_pseudodata(**common_dict)

In [ ]:
dat_reps = pd.concat(
    [i.pseudodata.reindex(prior_theorypreds_central.index) for i in pseudodata], axis=1
)

In [ ]:
invcov = np.linalg.inv(C + S)
delta_T_tilde = -S_hat @ invcov @ (mean_prediction - dat_reps.mean(axis=1))
P_tilde = S_hat.T @ invcov @ X @ invcov @ S_hat + S_tilde - S_hat.T @ invcov @ S_hat
pred = alphas_central + delta_T_tilde
unc = np.sqrt(P_tilde)
print(rf"Prediction for $\alpha_s$: {pred:.4f} ± {unc:.4f}")